In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
# from lightgbm.sklearn import LGBMRegressor
# import lightgbm as lgb

In [2]:
#피처 다합쳐진 데이터프레임 불러오기(드라이브 상에 TRAIN(소수점포함) 파일 불러오면 됨)
bm_ci_search_covid_consume = pd.read_csv('./train.csv')
bm_ci_search_covid_consume.drop(['Unnamed: 0'],axis=1,inplace=True)
bm_ci_search_covid_consume

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,year,FLC,month,AMT,out_home,diff,검색량(=100),confirmed,소비자심리지수
0,강원,건강보조식품 소매업,강원,1020s,2019,family,2,216200,1,-0.106782,0.02146,0.0,97.1
1,강원,건강보조식품 소매업,강원,1020s,2019,single,2,1517000,1,-0.106782,0.02146,0.0,97.1
2,강원,건강보조식품 소매업,강원,3060s,2019,family,2,144433371,1,-0.106782,0.02146,0.0,97.1
3,강원,골프장 운영업,강원,1020s,2019,family,2,1629420,1,0.034461,0.02146,0.0,97.1
4,강원,골프장 운영업,강원,1020s,2019,single,2,8254950,1,0.034461,0.02146,0.0,97.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265430,서울,정기 항공 운송업,충북,1020s,2020,single,4,3017380,0,-0.862732,40.21097,183.0,72.4
265431,서울,정기 항공 운송업,충북,3060s,2020,family,4,37190900,0,-0.862732,40.21097,183.0,72.4
265432,인천,정기 항공 운송업,충북,1020s,2020,single,4,54700,1,-0.862732,40.21097,29.0,72.4
265433,제주,정기 항공 운송업,충북,1020s,2020,single,4,480500,1,-0.862732,40.21097,4.0,72.4


In [3]:
#home 분리(거주지, 사용지역 같은 데이터)
home_condition = bm_ci_search_covid_consume['CARD_SIDO_NM']==bm_ci_search_covid_consume['HOM_SIDO_NM']
home = bm_ci_search_covid_consume[home_condition]
home.to_csv('train_home.csv', encoding='utf-8-sig')

In [4]:
#out 분리(거주지, 사용지역 다른 데이터)
out_condition = bm_ci_search_covid_consume['CARD_SIDO_NM']!=bm_ci_search_covid_consume['HOM_SIDO_NM']
out = bm_ci_search_covid_consume[out_condition]
out.to_csv('train_out.csv', encoding='utf-8-sig')

# 19년 234월 20년 234월 차이값 계산

In [ ]:
#2019년 234월 데이터 뽑아내기
year2019 = home['year']==2019
month2 = home['month']==2
month3 = home['month']==3
month4 = home['month']==4

year2019_ = out['year']==2019
month2_ = out['month']==2
month3_ = out['month']==3
month4_ = out['month']==4

home_2019 = home[year2019&(month2|month3|month4)]
out_2019 = out[year2019_&(month2_|month3_|month4_)]

In [ ]:
#2020년 234월 데이터 뽑아내기
year2020 = home['year']==2020
month2 = home['month']==2
month3 = home['month']==3
month4 = home['month']==4

year2020_ = out['year']==2020
month2_ = out['month']==2
month3_ = out['month']==3
month4_ = out['month']==4

home_2020 = home[year2020&(month2|month3|month4)]
out_2020 = out[year2020_&(month2_|month3_|month4_)]

In [ ]:
#기존 데이터의 피처가 같은 것끼리 매칭시키기 위해 merge
home_merge = pd.merge(home_2019,home_2020,on=['CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','FLC','month'])
out_merge = pd.merge(out_2019,out_2020,on=['CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','FLC','month'])

In [ ]:
home_merge = home_merge.drop(['year_x','year_y','out_home_x','diff_x','검색량(=100)_x','confirmed_x','소비자심리지수_x'],axis=1)
out_merge = out_merge.drop(['year_x','year_y','out_home_x','diff_x','검색량(=100)_x','confirmed_x','소비자심리지수_x'],axis=1)

#2019년 2020년 AMT 차이값 계산
home_merge['AMT_DIFF'] = (home_merge['AMT_x']-home_merge['AMT_y'])
out_merge['AMT_DIFF'] = (out_merge['AMT_x']-out_merge['AMT_y'])
# home_merge['AMT_DIFF'] = home_merge['AMT_DIFF'].map(log)
# out_merge['AMT_DIFF'] = out_merge['AMT_DIFF'].map(log)

home_merge = home_merge.drop(['AMT_x','AMT_y'],axis=1)
out_merge = out_merge.drop(['AMT_x','AMT_y'],axis=1)

In [ ]:
#이상치 범위 조정
home_merge = home_merge[home_merge['AMT_DIFF']<10000000]
home_merge = home_merge[-10000000<home_merge['AMT_DIFF']]
out_merge = out_merge[out_merge['AMT_DIFF']<1000000]
out_merge = out_merge[-1000000<out_merge['AMT_DIFF']]

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#타겟값 형태파악
sns.distplot(out_merge['AMT_DIFF'])

# 모델 성능확인

In [ ]:
#AMT차이가 양수이면 그냥 로그씌우고 음수이면 -없애고 로그 씌우고 -붙이는 함수
#성능이 너무안좋아서 우선 음수인경우 0으로
# def log(amt_diff):
#     if amt_diff >= 0:
#         amt_diff = np.log1p(amt_diff)
#     else:
#         amt_diff = -np.log1p(-amt_diff)
#     return amt_diff

In [ ]:
#target값 로그변환

# home_merge['AMT_DIFF'] = home_merge['AMT_DIFF'].map(log)
# out_merge['AMT_DIFF'] = out_merge['AMT_DIFF'].map(log)

#명목형 변수 원핫인코딩
home_dummy = pd.get_dummies(home_merge,columns=['CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','FLC','month'])
out_dummy = pd.get_dummies(out_merge,columns=['CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','FLC','month'])

In [ ]:
out_dummy

In [ ]:
#RMSLE함수 정의
def get_rmsle_home(model): 
    pred = model.predict(Xhome_test)
    squared_error = (np.log((yhome_test+1)/(pred+1)))** 2
    rmsle = np.sqrt(np.mean(squared_error))
    print(rmsle)
    return rmsle

def get_rmsle_out(model): 
    pred = model.predict(Xout_test)
    squared_error = (np.log((yout_test+1)/(pred+1)))** 2
    rmsle = np.sqrt(np.mean(squared_error))
    print(rmsle)
    return rmsle

# def get_rmsles(models):
#     rmsles = [ ]
#     for model in models:
#         rmsle = get_rmsle(model)
#         rmsles.append(rmsle)
#     return rmsles

In [ ]:
from sklearn import preprocessing

#인풋데이터(피처) 생성
home_features = home_dummy.drop('AMT_DIFF',axis=1, inplace=False)
out_features = out_dummy.drop('AMT_DIFF',axis=1, inplace=False)

#인풋데이터에 정규화 적용
min_max_scaler = preprocessing.MinMaxScaler()
home_scaled = min_max_scaler.fit_transform(home_features)
out_scaled = min_max_scaler.fit_transform(out_features)

home_features = pd.DataFrame(home_scaled)
out_features = pd.DataFrame(out_scaled)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#타겟값 생성
home_target = home_dummy['AMT_DIFF']
out_target = out_dummy['AMT_DIFF']

#학습,테스트 데이터 분리
Xhome_train, Xhome_test, yhome_train, yhome_test = train_test_split(home_features, home_target, test_size=0.2, random_state=0)
Xout_train, Xout_test, yout_train, yout_test = train_test_split(out_features, out_target, test_size=0.2, random_state=0)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

# Lasso
lasso_home = Lasso()
lasso_out = Lasso()

lasso_home.fit(Xhome_train, yhome_train)
lasso_out.fit(Xout_train, yout_train)

In [ ]:
# RamdomForestRegressor

rf_home = RandomForestRegressor(random_state=0)
rf_out = RandomForestRegressor(random_state=0)

rf_home.fit(Xhome_train, yhome_train)
rf_out.fit(Xout_train, yout_train)

In [ ]:
#차이값에 대한 rmsle결과 확인

get_rmsle_out(lasso_out)
get_rmsle_home(lasso_home)

get_rmsle_out(rf_out)
get_rmsle_home(rf_home)

In [ ]:
def rmsle(estimator, X, y0):
    pred = estimator.predict(X)
    squared_error = (np.log((y0+1)/(pred+1)))** 2
    rmsle = np.sqrt(np.mean(squared_error))
    if math.isnan(rmsle):
        print("this is a nan")

    return -rmsle

In [ ]:
# RandomForest 그리드 서치

rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
pipe_rf = Pipline([('scaler', StandardScaler()), ('reg', RandomForestRegressor(random_state=0))])

param_grid = [{'scaler':[StandardScaler()], 'n_estimators': [500], 'max_depth':[4,6,8,10], 'min_samples_leaf':[2,4,6], 'min_samples_split':[6,8,10]}]

rf_home = GridSearchCV(estimator=pipe_rf, param_grid = param_grid, cv=5, scoring = rmsle, n_jobs=-1)
rf_out = GridSearchCV(estimator=pipe_rf, param_grid = param_grid, cv=5, scoring = rmsle, n_jobs=-1)

rf_home.fit(home_features, home_target)
rf_out.fit(out_features, out_target)

In [ ]:
print('rf_home')
print(rf_home.best_score_)
print(rf_home.best_params_)

In [ ]:
print('rf_out')
print(rf_out.best_score_)
print(rf_out.best_params_)

In [ ]:
# Lasso 그리드 서치

rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
pipe_lasso = Pipeline([('scaler', StandardScaler()), ('reg', Lasso())])

param_range = [1000000,100000000]
param_grid = [{'scaler': [StandardScaler()],'reg__alpha': param_range}]


lasso_home = GridSearchCV(estimator=pipe_lasso, param_grid=param_grid,
                   scoring=rmsle,cv=5, n_jobs=-1)
lasso_out = GridSearchCV(estimator=pipe_lasso, param_grid=param_grid,
                   scoring=rmsle,cv=5, n_jobs=-1)

lasso_home.fit(home_features, home_target)
lasso_out.fit(out_features, out_target)

In [ ]:
print('lasso_home')
print(lasso_home.best_score_)
print(lasso_home.best_params_)

In [ ]:
print('lasso_out')
print(lasso_out.best_score_)
print(lasso_out.best_params_)

# 지금 컴퓨터에 문제가 있어서 수정 후에 랜덤포레스트 부분을 다시 못 돌려봤는데 혹시 오류 나면 말씀해주세요 ㅠㅠ